# This notebook implements the routine for flight-path calibration

        What it should be done:
        - read the theoretical spectrum
        - read the calibration sample
        - be able to convert from TOF to lambda and back
        - the peak positions are detected and single edge fitting is applied
        - calculate L, T0 or deltaL, which are the results of the calibration 
        
        time to lambda convertion:
            lambda = h/mL (t-t0)
            where:lambda = wavelength [A] (A=0.1nm = 1e-10m)
                    h = Planck's constant: 6.62607004 × 10-34 m^2 kg / s
                    m = Neutron mass [kg]: 1.674 927 471 x 10-27 kg 
                    L = total flight path [m]
                    t = time of flight [s]
        
       Convertion between time of flight and neutron wavelenght can be done after flight-path calibration: the value of the distance between the source and the transmission detector L0  and the time delay of the source trigger DT0, is obtained by linear regression to the function  lambda = h/mL (t-t0), or t= lambda(mL/h)+t0, with h= Planck constant
        
        txt files used as inputs are extracted from nxsplotter
        

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
from astropy.io import fits
import os, fnmatch
from os import listdir

import scipy.signal
print(scipy.__version__)
from scipy.signal import find_peaks

from ipywidgets import widgets
import IPython.display as Disp
import cv2

import AdvancedBraggEdgeFitting
from TOF_routines import find_first, find_last, find_nearest
from TOF_routines import tof2l, l2tof

import ipywe.fileselector
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

## Give as input the filename the theoretical spectrum (y-axis) and lambda (x-axis)
### here they are txt files extracted from nxsplotter

In [ ]:
# Select theoretical spectrum file (txt)
# alternative:
# file_s_the = input('Select theoretical spectrum file (txt): ')
# ex: /home/carminati_c/git/ToFImaging/scripts/data/alpha.txt
fsel_file_spectrum= ipywe.fileselector.FileSelectorPanel(instruction='Select txt file for the reference spectrum', start_dir='.', type='file')
fsel_file_spectrum.show()

In [ ]:
file_s_the = fsel_file_spectrum.selected
# ex: data/alpha.txt
print(file_s_the)

In [ ]:
# Select lambda txt file for the reference spectrum 
# alternative:
# file_l_th  =input('Select corresponding lambda (txt): ')
# ex: /home/carminati_c/git/ToFImaging/scripts/data/lambda.txt
fsel_file_lambda= ipywe.fileselector.FileSelectorPanel(instruction='Select txt file for the reference lambda', start_dir='.', type='file')
fsel_file_lambda.show()

In [ ]:
# Select corresponding lambda
file_l_th  = fsel_file_lambda.selected
print(file_l_th)
# ex: data/lambda.txt

In [ ]:
mylambda = np.genfromtxt(file_l_th, usecols=0)
myspectrum = np.genfromtxt(file_s_the, usecols=0)

In [ ]:
# Select list of corresponding hkls
# alternative: file_hkl = input()
# ex. data/alphaFe_hkl

fsel_file_hkl = ipywe.fileselector.FileSelectorPanel(instruction='Select txt file for the hkl list', start_dir='.', type='file')
fsel_file_hkl.show()

In [ ]:
file_hkl = fsel_file_hkl.selected
dhkl_alphaFe = np.genfromtxt(file_hkl, usecols=4)
h_alphaFe= np.genfromtxt(file_hkl, usecols=0)
k_alphaFe= np.genfromtxt(file_hkl, usecols=1)
l_alphaFe= np.genfromtxt(file_hkl, usecols=2)


In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
plt.plot(mylambda, myspectrum, label=r'$\alpha$ Fe, Martensite')

for i in range(0,7):
    plt.plot(np.array([2*dhkl_alphaFe[i],2*dhkl_alphaFe[i]]), np.array([7,22]), '--k')
    mytext = '('+str((h_alphaFe[i]).astype(int))+','+str(k_alphaFe[i].astype(int))+','+str(l_alphaFe[i].astype(int))+')'
    plt.text(2*dhkl_alphaFe[i]-0.05,24, mytext, rotation=45, color='k')
    plt.text(2*dhkl_alphaFe[i]-0.05,6.5, str(2*dhkl_alphaFe[i]), rotation=45, color='k')

plt.legend(loc='upper right')
plt.xlim(1,6.5)
plt.ylim(5,25)
plt.ylabel('total cross section [barn]')
plt.xlabel('wavelength [A]')
plt.title('Theoretical Fe total cross section and lattice parameters')

print('---- Peak positions given by the lattice parameters ----')
print(dhkl_alphaFe*2)

## Read the measured spectra file for the TOF calibration sample scan
### spectra txt file

In [ ]:
# Select spectra txt file
# alternative :
# spectra_txt_file = input("Select spectra file (txt): ")
# ex: /media/carminati_c/Data2/IMAT_Nov2018/02_HighStats_radio_1hruns/samples_after_reboot_OC_Fiji/IMAT00010420_HighStats_Radio_1hruns_000_Spectra.txt

fsel_spectra = ipywe.fileselector.FileSelectorPanel(instruction='Select spectra txt file', start_dir='.', type='file')
fsel_spectra.show()

In [ ]:
spectra_txt_file = fsel_spectra.selected
mycaltof = np.genfromtxt(spectra_txt_file, usecols=0)
myhist = np.genfromtxt(spectra_txt_file, usecols=1) #this is the cumulative histogram of the raw data (before the overlap correction)



plt.plot(mycaltof,myhist)
plt.show()

## Read the calibration datasets

### TOF sample directory

In [ ]:
# here copy the path of the sample data
# alternative:
# pathdata = input("Copy the path for the sample data: ")
#ex: /media/carminati_c/Data2/IMAT_Nov2018/02_HighStats_radio_1hruns/Samples_AfterReboot_Corrected/

fsel_data= ipywe.fileselector.FileSelectorPanel(instruction='select path for the sample data', start_dir='.', type='directory')
fsel_data.show()

In [ ]:
pathdata = fsel_data.selected
print(pathdata)

In [ ]:
# Here copy the path of the open beam data
# alternative:
# pathOB = input('Copy the path for the open beam data: ')
# ex: /media/carminati_c/Data2/IMAT_Nov2018/02_HighStats_radio_1hruns/Flat_AfterReboot_Corrected/

fsel_ob= ipywe.fileselector.FileSelectorPanel(instruction='select path for the ob data', start_dir='.', type='directory')
fsel_ob.show()


In [ ]:
pathOB = fsel_ob.selected

In [ ]:
#sort the files
myfiles = fnmatch.filter(listdir(pathdata),'*.fits')
coll_files = sorted(myfiles)
obfiles = fnmatch.filter(listdir(pathOB),'*.fits') # here there are several OB folders
coll_ob = sorted(obfiles)


In [ ]:
class bbox_select():
    %matplotlib notebook 


    def __init__(self,im):
        self.im = im
        self.selected_points = []
        self.fig,ax = plt.subplots()
        self.img = ax.imshow(self.im.copy(), cmap='gray', vmin=0, vmax=10)
        self.ka = self.fig.canvas.mpl_connect('button_press_event', self.onclick)
        disconnect_button = widgets.Button(description="Disconnect mpl")
        Disp.display(disconnect_button)
        disconnect_button.on_click(self.disconnect_mpl)


        
    def poly_img(self,img,pts):
        pts = np.array(pts, np.int32)
        pts = pts.reshape((-1,1,2))
        color = (255,255,0)
        cv2.polylines(img,[pts],True,color,7)
                      
#         cv2.Rectangle(img, pt1, pt2, color, thickness=7)
#         pts = [[pt1],[pt2]]
        return img

    def onclick(self, event):
        display(str(event))
        self.selected_points.append([event.xdata,event.ydata])
        if len(self.selected_points)>1:
            self.fig
            self.img.set_data(self.poly_img(self.im.copy(),self.selected_points))
    def disconnect_mpl(self,_):
        self.fig.canvas.mpl_disconnect(self.ka)


In [ ]:
print("select 4 points for a rectangular ROI, then click Disconnect mpl")
filename = pathdata +'/'+ coll_files[0]
im = fits.open(filename)
bs = bbox_select(im[0].data)
# here I have to change the fact that the ROI is blue like the image

In [ ]:
bs.selected_points


### Plot of the region used for calibration 

In [ ]:
roi_cal = np.array([0,0, 10,10])
sel_points = np.squeeze(np.array([bs.selected_points],'int'))
print(sel_points)
roi_cal[0] = sel_points[0][0]
roi_cal[1] = sel_points[0][1]
roi_cal[2] = sel_points[2][0]
roi_cal[3] = sel_points [2][1]

plt.figure()
plt.imshow(im[0].data[roi_cal[1]:roi_cal[3],roi_cal[0]:roi_cal[2]]) #this is the area that I want to study


### Load data and plot calibration spectrum

In [ ]:
cal_spectrum = np.zeros(len(coll_files))
std_spectrum = np.zeros(len(coll_files))
cal_ob = np.zeros(len(coll_files))
std_ob = np.zeros(len(coll_files))
ori_hist = np.zeros(len(coll_files))
collImg= np.zeros([512,512,len(coll_files)])
collOB = np.zeros([512,512, len(coll_files)])

for i in range(0, len(coll_files)):
    
    curr_img = (fits.open(pathdata+'/'+coll_files[i])[0].data[roi_cal[1]:roi_cal[3],roi_cal[0]:roi_cal[2]]).astype(float)
    curr_ob =(fits.open(pathOB+'/'+coll_ob[i])[0].data[roi_cal[1]:roi_cal[3],roi_cal[0]:roi_cal[2]]).astype(float)
    cal_spectrum[i] = np.sum(curr_img[~np.isnan(curr_img) & ~np.isinf(curr_img)])
    cal_ob[i]= np.sum(curr_ob[~np.isnan(curr_ob) & ~np.isinf(curr_ob)])
    std_spectrum[i] = np.std(curr_img/curr_ob)
#     std_ob[i] = np.std(curr_ob[~np.isnan(curr_ob) & ~np.isinf(curr_ob)])
    
    
    

In [ ]:
# Plot calibration spectrum 
cal_spectrum_norm = cal_spectrum/cal_ob
plt.figure()
plt.plot(mycaltof, (cal_spectrum/cal_ob))
plt.title('Calibration spectrum, Fe')
plt.xlabel('TOF [s]')
plt.ylabel('Transmission I/$I_{0}$')
plt.show()

In [ ]:
tof_ranges=np.zeros((6,2))

tof_ranges[5,0] = 0.05
tof_ranges[5,1] = 0.08

tof_ranges[4,0] =  0.038
tof_ranges[4,1] =  0.05

tof_ranges[3,0] = 0.031
tof_ranges[3,1] = 0.036

tof_ranges[2,0] = 0.0275
tof_ranges[2,1] =  0.031

tof_ranges[1,0] = 0.0245
tof_ranges[1,1] =  0.0275

tof_ranges[0,0] = 0.0205
tof_ranges[0,1] =  0.0235


peaks, _ = find_peaks(-cal_spectrum_norm, width=15)
plt.figure()
plt.plot(mycaltof, cal_spectrum_norm)
plt.plot(mycaltof[peaks],cal_spectrum_norm[peaks],'x', markeredgewidth=3)
for i in range(0,6):
    plt.plot((tof_ranges[i,0], tof_ranges[i,0]),(0.1,0.8), 'k-')
    plt.plot((tof_ranges[i,1], tof_ranges[i,1]),(0.1,0.8), 'k-')

plt.title('Spectrum for Fe powder')
plt.xlabel('TOF[s]')
plt.ylabel('Transmission I/I$_{0}$')
plt.show()
print(mycaltof[peaks])
print(peaks)


### here I loop over all selected peaks 

In [ ]:

print(peaks) # here I see which are the positions of the peaks in the cal spectrum


#in TOF [s!]
est_sigma = 0.0001
est_alpha = np.array([0.0001,0.0001,0.0001,0.0001,0.0001,0.0015]) # This takes into account that the alpha values as a pseudo-sigmoid behaviour
CalPoints = np.zeros(6)
RefPoints = np.zeros(6)

RefPoints[0]=dhkl_alphaFe[6]*2

#Here I exclude the peak for hkl (2,2,2)
for i in range(1, 6):
    RefPoints[i]=dhkl_alphaFe[5-i]*2

print('--- RefPoints obtained from lattice parameters ---')
print(RefPoints)
print('number of peaks: ',len(peaks))

computed_alpha = np.zeros(len(peaks))

for i in range(0, len(peaks)):
# for i in range(len(peaks)-1, len(peaks)):
    
    print(tof_ranges[i,:])
    myrange = np.array([find_nearest(mycaltof, tof_ranges[i,0]), find_nearest(mycaltof,tof_ranges[i,1])])
    print(myrange)
    print(peaks[i])
    print(tof_ranges[i,:])
    print(mycaltof[peaks[i]])
            

    print('-----------Fitting calibration Bragg Edge--------------')
    results_cal = AdvancedBraggEdgeFitting.AdvancedBraggEdgeFitting(cal_spectrum_norm, myrange, mycaltof, peaks[i], est_sigma, est_alpha[i], True, False, False,True)
    print('-----------Results of fitting: Edge Position------------')
    print(results_cal['t0'])

#     CalPoints[i] = mycaltof[myrange[0]+int(results_cal['t0'])]
    CalPoints[i] = results_cal['t0']
    computed_alpha[i] = results_cal['sigma']

    print(CalPoints)
#     print(RefPoints)

print('computed moderator decay:')
print(computed_alpha)
print(RefPoints)
print(CalPoints)
print(i)

## I will then fit the calculated TOF to the theoretical lambda, x= RefPoints (theoretical lambda) y=mycaltof (computed TOF) 

In [ ]:
#Calibration on the lattice parameters value
print(RefPoints) # theoretical lambda
print(CalPoints) # calculated TOF
h=6.62607004e-34 #Planck constant [m^2 kg / s]
m=1.674927471e-27 #Neutron mass [kg]

z= np.polyfit( CalPoints,RefPoints,1)
print('Calibration parameters:')
print('flight path L:', (h/(m*z[0]))/1e-10, 'm')
print('T0:', z[1]/z[0] ,'s')
print(z)

plt.figure()
plt.plot(CalPoints,RefPoints,'ob')
plt.plot( mycaltof, mycaltof*z[0]+z[1],'-g')
plt.ylabel('wavelength theoretical [A]')
plt.xlabel('ToF fitted [s]')
plt.title('Calibration plot')
